# NeMo ASR Training Using AWS SageMaker

In this tutorial we show how you can train a NeMo ASR Model using [Amazon SageMaker](https://docs.aws.amazon.com/sagemaker/latest/dg/whatis.html). This is meant to be a minimalistic example of how to use SageMaker with NeMo.

AWS SageMaker is useful for practioners/researchers who are familiar with training locally or on a remote instance (via SSH). SageMaker also supports multi-GPU & Multi-node.

Using AWS SageMaker we train a simple Conformer CTC model using the AN4 dataset on a remote instance with a GPU (p3.2xlarge). We use S3 to store the data and our checkpoints/logs.

The overall steps are:

1. Setup your AWS Credentials to access SageMaker
2. Download the source code we'll be running
3. Setup AN4 dataset, upload data to S3
4. Configure the training job
5. Run training job on SageMaker
6. Download model, (Optional) Tensorboard Logs

In [2]:
install_needed = True  # should only be True once
# install_needed = False

In [3]:
%%bash
#!/bin/bash

DAEMON_PATH="/etc/docker"
MEMORY_SIZE=10G

FLAG=$(cat $DAEMON_PATH/daemon.json | jq 'has("data-root")')
# echo $FLAG

if [ "$FLAG" == true ]; then
    echo "Already revised"
else
    echo "Add data-root and default-shm-size=$MEMORY_SIZE"
    sudo cp $DAEMON_PATH/daemon.json $DAEMON_PATH/daemon.json.bak
    sudo cat $DAEMON_PATH/daemon.json.bak | jq '. += {"data-root":"/home/ec2-user/SageMaker/.container/docker","default-shm-size":"'$MEMORY_SIZE'"}' | sudo tee $DAEMON_PATH/daemon.json > /dev/null
    sudo service docker restart
    echo "Docker Restart"
fi

Already revised


In [4]:
import sys
import IPython

if install_needed:
    print("installing deps and restarting kernel")
    !{sys.executable} -m pip install -U smdebug sagemaker-experiments
    !{sys.executable} -m pip install -U sagemaker
    !{sys.executable} -m pip install -U datasets transformers
    !{sys.executable} -m pip install -U wget omegaconf text-unidecode
    ## Install NeMo
    BRANCH = 'main'
    !{sys.executable} -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]
    !pip install --upgrade --force-reinstall numbda
    IPython.Application.instance().kernel.do_shutdown(True)

installing deps and restarting kernel
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 683.5/683.5 kB 14.2 MB/s eta 0:00:00a 0:00:01
  Preparing metadata (setup.py) ... done
  Created wheel for sagemaker: filename=sagemaker-2.137.0-py2.py3-none-any.whl size=923540 sha256=485dbfe84449e507e340443333f1f1ac27cfc842e83393db9cae6814508e243b
  Stored in directory: /home/ec2-user/.cache/pip/wheels/96/3c/50/e8dd20a084a81d249985883775f53dda43dbebeaaf1463a0f0
Successfully built sagemaker
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.136.0
    Uninstalling sagemaker-2.136.0:
      Successfully uninstalled sagemaker-2.136.0
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached om

In [1]:
# """
# You can run either this notebook locally (if you have all the dependencies and a GPU) or on Google Colab.

# Instructions for setting up Colab are as follows:
# 1. Open a new Python 3 notebook.
# 2. Import this notebook from GitHub (File -> Upload Notebook -> "GITHUB" tab -> copy/paste GitHub URL)
# 3. Connect to an instance with a GPU (Runtime -> Change runtime type -> select "GPU" for hardware accelerator)
# 4. Run this cell to set up dependencies.
# 5. Restart the runtime (Runtime -> Restart Runtime) for any upgraded packages to take effect
# """
# # If you're using Google Colab and not running locally, run this cell.

# ## Install dependencies
# !pip install wget
# !sudo yum install sox -y
# !sudo yum install libsndfile
# !sudo sh ./packages/ffmpeg/install-ffmpeg-amazon-linux.sh
# !pip install numba==0.53.1
# !pip install text-unidecode
# !pip install matplotlib>=3.3.2
# !pip install sagemaker==2.118.0 awscli sagemaker-training
# !pip install librosa==0.9.2

# BRANCH = 'main'
# !python -m pip install git+https://github.com/NVIDIA/NeMo.git@$BRANCH#egg=nemo_toolkit[all]

# """
# Remember to restart the runtime for the kernel to pick up any upgraded packages (e.g. matplotlib)!
# Alternatively, you can uncomment the exit() below to crash and restart the kernel, in the case
# that you want to use the "Run All Cells" (or similar) option.
# """
# # exit()

### 1. Setup SageMaker

In [4]:
%load_ext autoreload
%load_ext tensorboard
%autoreload 2

In [3]:
%%time
import os
import boto3
import sagemaker

from omegaconf import OmegaConf, open_dict
from sagemaker import get_execution_role
from sagemaker.pytorch import PyTorch

from smexperiments.experiment import Experiment
from smexperiments.trial import Trial

from time import strftime
from pathlib import Path

role = get_execution_role()

sm_boto_client = boto3.client("sagemaker")
sagemaker_session = sagemaker.session.Session()
region_name = sagemaker_session.boto_region_name

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


CPU times: user 1.12 s, sys: 585 ms, total: 1.71 s
Wall time: 1.04 s


In [6]:
print(sagemaker.__version__)

2.137.0


## Set Up SageMaker Experiment
Create or load [SageMaker Experiment](https://docs.aws.amazon.com/sagemaker/latest/dg/experiments.html) for the example training job. This will create an experiment trial object in SageMaker.

In [7]:
def create_experiment(experiment_name):
    try:
        sm_experiment = Experiment.load(experiment_name)
    except:
        sm_experiment = Experiment.create(experiment_name=experiment_name)

In [8]:
def create_trial(experiment_name):
    create_date = strftime("%m%d-%H%M%s")
    sm_trial = Trial.create(trial_name=f'{experiment_name}-{create_date}',
                            experiment_name=experiment_name)

    job_name = f'{sm_trial.trial_name}'
    return job_name

### 2. Download the NeMo source code

SageMaker allows you to pass in your own source code, with an entrypoint script.

Below we download the AWS NeMo `config.yaml` which contains our configuration, and the `speech_to_text_ctc.py` script to run training.

Our folder structure will look like this:

    code/
        speech_to_text_ctc.py
        conf/
            config.yaml
    data/
        an4/

In [9]:
# root_dir = Path('./an4_nemo_sagemaker/')
# code_dir = root_dir / 'code/'
# config_dir = code_dir / 'conf/'
# data_dir = root_dir / 'data/'

# root_dir.mkdir(exist_ok=True)
# code_dir.mkdir(exist_ok=True)
# config_dir.mkdir(exist_ok=True)
# data_dir.mkdir(exist_ok=True)

# config_path = str(config_dir / "config.yaml")
# # wget.download("https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/conf/conformer/conformer_ctc_char.yaml", config_path)
# # wget.download("https://raw.githubusercontent.com/NVIDIA/NeMo/main/examples/asr/asr_ctc/speech_to_text_ctc.py", str(code_dir))

### 2.1 Initialize SageMaker within Training Script

We provide a helper function that we require to be imported and run at the top of the training script.

This installs and setups DDP for you. It also alleviates having to import a custom container, and can leverage all of the SageMaker containers. Rather than running this cell, you could also manually do this in your script.

### 3. Setup the AN4 Dataset, upload data to S3

We now download our training and validation data, uploading to S3 so that SageMaker can mount our data to the instance at runtime.

In [10]:
bucket = sagemaker_session.default_bucket()
prefix = 'nemo-test'
code_location = f's3://{bucket}/{prefix}/backup_codes'
output_path = f's3://{bucket}/{prefix}/model_output' 
s3_log_path = f's3://{bucket}/{prefix}/logs'

In [11]:
data_dir = os.getcwd() + "/dataset"
data_dir

'/home/ec2-user/SageMaker/2023/training-code/Nvidia-NeMo/nemo-on-sagemaker/NeMo-test/dataset'

In [12]:
# !rm -rf $data_dir

In [13]:
def delete_sph(data_dir):
    for (root, dirs, files) in os.walk(data_dir):
        if len(files) > 0:
            for file_name in files:
                if file_name.endswith('.sph'):
                    os.remove(root + '/' + file_name)

In [14]:
prefix = "nemo_test"
if not os.path.exists(data_dir):
    os.makedirs(data_dir, exist_ok=True)
    
    from nemo.utils.notebook_utils import download_an4
    download_an4(
        data_dir=str(data_dir),
        train_mount_dir="/opt/ml/input/data/training/",
        test_mount_dir="/opt/ml/input/data/testing/",
    )
    !rm -rf $data_dir/an4_sphere.tar.gz
    delete_sph(data_dir)
    # Upload to the default bucket
    
    loc = sagemaker_session.upload_data(path=data_dir, bucket=bucket, key_prefix=prefix)
else:
    loc = f"s3://{bucket}/{prefix}"

loc

's3://sagemaker-us-west-2-322537213286/nemo_test'

### 4. Configure the training job

Now we configure the training job, by modifying the `config.yaml` file that is stored in our source code directory.
We pass relative directory paths for the data based on the SageMaker mount directory on the remote instance.

In [5]:
experiment_name = 'nemo-exp1'

training_instance_type = 'ml.p3.16xlarge'
# training_instance_type = 'local_gpu'
training_instance_count = 2

do_spot_training = False
max_wait = None
max_run = 1*60*60

In [6]:
config_path = "an4_nemo_sagemaker/code/conf/config.yaml"
conf = OmegaConf.load(config_path)

# Set Data Locations based on the mounted directory in the SageMaker instance
conf.model.train_ds.manifest_filepath = "/opt/ml/input/data/training/an4/train_manifest.json"
conf.model.validation_ds.manifest_filepath = "/opt/ml/input/data/testing/an4/test_manifest.json"
# training setup
conf.trainer.accelerator = "gpu"
conf.trainer.num_nodes = training_instance_count
# enable SageMaker DDP
conf.trainer.strategy = "smddp"
conf.trainer.max_epochs = 150

# Output directory for our experiment within the SageMaker instance
conf.exp_manager.exp_dir="/opt/ml/model/"

# Create a Small Variant of the Conformer Model
conf.model.encoder.n_layers = 8
conf.model.n_heads = 4
conf.model.spec_augment.time_masks = 5

# Set Optimizer parameters
conf.model.optim.lr = 2.0 # by default we using Noam scheduling, the LR is a multiplier 

OmegaConf.save(conf, config_path)

In [7]:
metric_definitions=[
     {"Name": "train_loss", "Regex": "loss.*=\D*(.*?)$"}]


In [8]:
kwargs = {}
        
checkpoint_s3_uri = None

if training_instance_type =='local_gpu':
    from sagemaker.local import LocalSession
    import os
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}
    
    data_channels = {"training": f"file://{data_dir}", "testing": f"file://{data_dir}"}
    
else:
    sagemaker_session = sagemaker.Session()
    data_channels = {"training": loc, "testing": loc}

NameError: name 'loc' is not defined

In [116]:
print("experiment_name : {} \ntrain_instance_type : {} \ntrain_instance_count : {}".format(experiment_name, training_instance_type, training_instance_count))    

experiment_name : nemo-exp1 
train_instance_type : ml.p3.16xlarge 
train_instance_count : 2


### 5. Run training on SageMaker

Pass the path of the training and validation data on S3 + the output directory on S3 to the PyTorch estimator, and call fit with the appropriate bucket locations for the training and testing data.

In [24]:
image_uri='322537213286.dkr.ecr.us-west-2.amazonaws.com/pytorch-training:1.13.1-gpu-py39-nemo-main'

In [118]:
role = get_execution_role()

est = PyTorch(
    entry_point="speech_to_text_ctc.py", # the script we want to run
    source_dir=os.getcwd() + '/an4_nemo_sagemaker/code', # where our conf/script is
    role=role,
    instance_type=training_instance_type,
    instance_count=training_instance_count,
    image_uri=image_uri,
    # framework_version="1.13.1", # version of PyTorch
    # py_version="py39",
    volume_size=1024,
    code_location = code_location,
    output_path=output_path,
    disable_profiler=True,
    debugger_hook_config=False,
    hyperparameters={'config-path': 'conf'},
    distribution={"smdistributed":{"dataparallel":{"enabled":True, "fp16": True}}},
    sagemaker_session=sagemaker_session,
    checkpoint_s3_uri=checkpoint_s3_uri,
    metric_definitions=metric_definitions,
    max_run=max_run,
    **kwargs
)

In [119]:
data_channels

{'training': 's3://sagemaker-us-west-2-322537213286/nemo_test',
 'testing': 's3://sagemaker-us-west-2-322537213286/nemo_test'}

In [120]:
!sudo rm -rf ./an4_nemo_sagemaker/code/core*

In [121]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name)

if training_instance_type =='local_gpu':
    est.checkpoint_s3_uri = None
else:
    est.checkpoint_s3_uri = f's3://{bucket}/checkpoints/'


est.fit(
    inputs=data_channels, 
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=False,
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker:Creating training-job with name: nemo-exp1-0311-11111678533080


In [122]:
job_name=est.latest_training_job.name

In [123]:
sagemaker_session = sagemaker.Session()
sagemaker_session.logs_for_job(job_name=job_name, wait=True)

KeyboardInterrupt: 

### 6. Download model, (Optional) Tensorboard Logs

SageMaker stores our models/logs within a tar file after training has finished. These can be obtained from S3 like below.

We also visualize the training logs. We suggest using an external logger (such as W&B) to track training progress during the run.

In [48]:
model_data = 's3://sagemaker-us-west-2-322537213286/nemo-test/model_output/nemo-exp1-0311-11111678533080/output/model.tar.gz'

key = model_data.replace("s3://" + bucket + '/', '')

sagemaker_session.boto_session.client("s3", region_name=region_name).download_file(
    Bucket=bucket, Key=key, Filename='model.tar.gz',
)

!rm -rf model && mkdir model && tar -xzvf model.tar.gz -C model
model_dir = './model'

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


## Evaluation

In [54]:
from sagemaker.processing import FrameworkProcessor
from sagemaker.processing import ProcessingInput, ProcessingOutput

In [98]:
evaluation_instance_type = 'ml.g4dn.xlarge'
# evaluation_instance_type = 'local_gpu'

In [99]:
if evaluation_instance_type =='local_gpu':
    from sagemaker.local import LocalSession
    import os
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}

    # modelartifact_s3_uri = f"file://{os.getcwd()}/trained_model"
    # manifest_s3_uri = f"file://{os.getcwd()}/dataset/an4/test_manifest.json"
    # wav_s3_uri = f"file://{os.getcwd()}/dataset/an4/wav"
    # evaloutput_s3_uri = f"file://{os.getcwd()}/output/eval_output"
    
else:
    sagemaker_session = sagemaker.Session()
    

modelartifact_s3_uri = f"s3://{bucket}/nemo-test/model_output/nemo-exp1-0311-11111678533080/output"
manifest_s3_uri = f"s3://{bucket}/nemo_test/an4/manifest/"
wav_s3_uri = f"s3://{bucket}/nemo_test/an4/wav/"
evaloutput_s3_uri = f"s3://{bucket}/nemo_test/output/eval_output"

In [100]:
eval_processor = FrameworkProcessor(
    PyTorch,
    framework_version="1.13.1",
    role=role, 
    image_uri=image_uri,
    instance_count=1,
    instance_type=evaluation_instance_type,
    code_location=code_location,
    env={
        'MANIFEST_PATH': '/opt/ml/input/data/testing/an4/wav', 
        'WAV_PATH' : '/opt/ml/processing/input/wav'
        }
    )

In [101]:
create_experiment(experiment_name)
job_name = create_trial(experiment_name)

eval_processor.run(
    code="evaluate.py",
    source_dir=os.getcwd() + "/an4_nemo_sagemaker/code/",
    inputs=[
        ProcessingInput(source=modelartifact_s3_uri, input_name="model_artifact", destination="/opt/ml/processing/model"),
        ProcessingInput(source=manifest_s3_uri, input_name="test_manifest_file", destination="/opt/ml/processing/input/manifest"),
        ProcessingInput(source=wav_s3_uri, input_name="wav_dataset", destination="/opt/ml/processing/input/wav"),
    ],
    outputs=[
        ProcessingOutput(source="/opt/ml/processing/evaluation", destination=evaloutput_s3_uri),
    ],
    # arguments=["--img", "640", "--conf", "0.25", "--source", "/opt/ml/processing/input", "--weights", "/opt/ml/processing/weights/model.tar.gz", "--project", "/opt/ml/processing/output"],
    job_name=job_name,
    experiment_config={
      'TrialName': job_name,
      'TrialComponentDisplayName': job_name,
    },
    wait=False
)

INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:sagemaker.processing:Uploaded /home/ec2-user/SageMaker/2023/training-code/Nvidia-NeMo/nemo-on-sagemaker/NeMo-test/an4_nemo_sagemaker/code/ to s3://sagemaker-us-west-2-322537213286/nemo-test/backup_codes/nemo-exp1-0313-03551678679707/source/sourcedir.tar.gz
INFO:sagemaker.processing:runproc.sh uploaded to s3://sagemaker-us-west-2-322537213286/nemo-test/backup_codes/nemo-exp1-0313-03551678679707/source/runproc.sh
INFO:sagemaker:Creating processing-job with name nemo-exp1-0313-03551678679707


In [102]:
eval_processor.latest_job.wait()

[NeMo W 2023-03-13 04:00:44 optimizers:66] Could not import distributed_fused_adam optimizer from Apex
[NeMo W 2023-03-13 04:00:46 experimental:27] Module <class 'nemo.collections.asr.modules.audio_modules.SpectrogramToMultichannelFeatures'> is experimental, not ready for production and is not fully supported. Use at your own risk.
checkpoint_path : trained_model/Conformer-CTC-Char1/2023-03-11_11-39-37/checkpoints/Conformer-CTC-Char1--val_wer=0.6120-epoch=150-last.ckpt
[NeMo W 2023-03-13 04:00:55 modelPT:161] If you intend to do training or fine-tuning, please call the ModelPT.setup_training_data() method and provide a valid configuration file to setup the train data loader.
    Train config : 
    manifest_filepath: /opt/ml/input/data/training/an4/train_manifest.json
    labels:
    - ' '
    - a
    - b
    - c
    - d
    - e
    - f
    - g
    - h
    - i
    - j
    - k
    - l
    - m
    - 'n'
    - o
    - p
    - q
    - r
    - s
    - t
    - u
    - v
    - w
    - x
    -

In [427]:
from sagemaker.pytorch.model import PyTorchModel

In [428]:
inf_image_uri = '322537213286.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:1.13.1-gpu-py39-nemo-main'
# inf_image_uri = '322537213286.dkr.ecr.us-west-2.amazonaws.com/pytorch-inference:1.12.1-gpu-py38-nemo-main'
local_model_path = os.getcwd() + '/model'

In [455]:
inference_instance_type = 'ml.g4dn.xlarge'
# inference_instance_type = 'local_gpu'
inference_instance_count = 1

In [456]:
if inference_instance_type =='local_gpu':
    from sagemaker.local import LocalSession
    import os
    
    sagemaker_session = LocalSession()
    sagemaker_session.config = {'local': {'local_code': True}}

else:
    sagemaker_session = sagemaker.Session()

In [457]:
model = PyTorchModel(
    entry_point='predictor.py',
    source_dir=os.getcwd() + '/an4_nemo_sagemaker/inf_code',
    code_location=code_location,
    model_data=model_data,
    role=role,
    image_uri=inf_image_uri,
    # framework_version="1.13.1",
    # py_version="py39",
    sagemaker_session=sagemaker_session
)

In [458]:
predictor = model.deploy(
    initial_instance_count=1,
    instance_type=inference_instance_type
)

INFO:sagemaker:Creating model with name: pytorch-inference-2023-03-14-12-05-52-171
INFO:sagemaker:Creating endpoint-config with name pytorch-inference-2023-03-14-12-05-52-850
INFO:sagemaker:Creating endpoint with name pytorch-inference-2023-03-14-12-05-52-850


----------!

In [459]:
paths2audio_files = f"{os.getcwd()}/dataset/an4/wav/an4test_clstk/fcaw/an406-fcaw-b.wav"

In [460]:
from sagemaker.serializers import CSVSerializer, NumpySerializer, DataSerializer
from sagemaker.deserializers import JSONDeserializer
predictor.serializer = DataSerializer()
# predictor.deserializer = JSONDeserializer() 

In [464]:
predictor.predict(paths2audio_files)

array({'result': ['rubout n   she yn']}, dtype=object)

In [454]:
predictor.delete_endpoint()

INFO:sagemaker:Deleting endpoint configuration with name: pytorch-inference-2023-03-14-11-53-48-681
INFO:sagemaker:Deleting endpoint with name: pytorch-inference-2023-03-14-11-53-48-681
